In [106]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import ast
from tqdm import tqdm

path = r'D:\Code\Traffic\Set1Part0'

def create_color_mask(image, lower_range, upper_range):
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    hsv_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2HSV)
    color_mask = cv2.inRange(hsv_image, lower_range, upper_range)
    kernel = np.ones((5, 5), np.uint8)
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, kernel)
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_CLOSE, kernel)
    return color_mask

def createBlueMask(image):
    lower_blue = np.array([90, 170, 60])
    upper_blue = np.array([125, 255, 255])
    return create_color_mask(image, lower_blue, upper_blue)

def createYellowMask(image):
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    return create_color_mask(image, lower_yellow, upper_yellow)

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    intersection_x1 = max(x1, x2)
    intersection_y1 = max(y1, y2)
    intersection_x2 = min(x1 + w1, x2 + w2)
    intersection_y2 = min(y1 + h1, y2 + h2)
    intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)
    area1 = w1 * h1
    area2 = w2 * h2
    iou = intersection_area / float(area1 + area2 - intersection_area)
    return iou

def is_rectangular(box, aspect_ratio_threshold=2.0):
    x, y, w, h = box
    aspect_ratio = float(w) / h
    return aspect_ratio > aspect_ratio_threshold or 1.0 / aspect_ratio > aspect_ratio_threshold

def combine_and_label_boxes(bboxes, labels, min_iou):
    combined_bboxes = []
    combined_labels = []
    used = [False] * len(bboxes)
    
    for i in range(len(bboxes)):
        if used[i]:
            continue
        x1, y1, w1, h1 = bboxes[i]
        label1 = labels[i]
        for j in range(i + 1, len(bboxes)):
            if used[j]:
                continue
            x2, y2, w2, h2 = bboxes[j]
            label2 = labels[j]
            iou = calculate_iou((x1, y1, w1, h1), (x2, y2, w2, h2))
            if iou >= min_iou:
                x1 = min(x1, x2)
                y1 = min(y1, y2)
                w1 = max(x1 + w1, x2 + w2) - x1
                h1 = max(y1 + h1, y2 + h2) - y1
                label1 = label1 if w1 * h1 >= w2 * h2 else label2
                used[j] = True
        if not is_rectangular((x1, y1, w1, h1)):
            combined_bboxes.append((x1, y1, w1, h1))
            combined_labels.append(label1)
    return combined_bboxes, combined_labels

def draw_and_label_boxes(image, mask, label, min_area, max_area):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bboxes = []
    labels = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area <= area <= max_area:
            x, y, w, h = cv2.boundingRect(cnt)
            bboxes.append((x, y, w, h))
            labels.append(label)
    combined_bboxes, combined_labels = combine_and_label_boxes(bboxes, labels, min_iou=0.02)
    for (x, y, w, h), label in zip(combined_bboxes, combined_labels):
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def zoom_image(image, scale=1.5):
    h, w = image.shape[:2]
    new_h, new_w = int(h * scale), int(w * scale)
    zoomed_image = cv2.resize(image, (new_w, new_h))
    return zoomed_image

def detect_traffic_signs_in_image(image, scale_factor_w, scale_factor_h, original_image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([255, 30, 255], dtype=np.uint8)

    lower_red1 = np.array([0, 100, 100], dtype=np.uint8)
    upper_red1 = np.array([5, 255, 255], dtype=np.uint8)
    
    lower_red2 = np.array([170, 100, 100], dtype=np.uint8)
    upper_red2 = np.array([180, 255, 255], dtype=np.uint8)

    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)

    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    combined_mask = cv2.bitwise_or(mask_white, mask_red)

    blurred = cv2.GaussianBlur(combined_mask, (5, 5), 0)

    kernel = np.ones((5, 5), np.uint8)
    combined_mask = cv2.morphologyEx(blurred, cv2.MORPH_OPEN, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)

    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_white, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours_red:
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) >= 5:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x * scale_factor_w), int(y * scale_factor_h))
            radius = int(radius)

            if 10 < radius < 80:
                cv2.circle(original_image, center, radius, (0, 0, 255), 2)

    for contour in contours_white:
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(contour)
            if 4000 < w * h < 7000:
                cv2.drawContours(original_image, [approx], 0, (0, 255, 0), 2)


def detect_traffic_signs_with_border(image_path):
    img = cv2.imread(image_path)
    hh, ww = img.shape[:2]

    zoomed_img = zoom_image(img, scale=3.0)

    blue_mask = createBlueMask(zoomed_img)
    yellow_mask = createYellowMask(zoomed_img)

    min_area = 1200
    max_area = 6800
    draw_and_label_boxes(zoomed_img, blue_mask, "traffic sign (blue)", min_area, max_area)
    draw_and_label_boxes(zoomed_img, yellow_mask, "traffic sign (yellow)", min_area, max_area)

    zoomed_h, zoomed_w = zoomed_img.shape[:2]
    scale_factor_h, scale_factor_w = hh / zoomed_h, ww / zoomed_w

    detect_traffic_signs_in_image(zoomed_img, scale_factor_w, scale_factor_h, img)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

annotations_file = r'D:\Code\Traffic\annotationsSet1.xlsx'
annotations_df = pd.read_excel(annotations_file)

def extract_annotations_for_image(image_name):
    image_annotations = annotations_df[annotations_df['image_name'] == image_name]
    annotations = []

    for index, row in image_annotations.iterrows():
        bbox = ast.literal_eval(row['bbox'])
        if bbox != [-1, -1, -1, -1]:
            x1, y1, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]) - int(bbox[0]), int(bbox[3]) - int(bbox[1])
            label = row['sign_type']
            annotations.append(((x1, y1, w, h), label))

    return annotations

tp_list = []
fp_list = []
fn_list = []
mean_ious = []
iou_threshold = 0.1

for root, dirs, files in os.walk(path):
    for filename in tqdm(files, desc="Processing Images"):
        img_path = os.path.join(root, filename)

        img = cv2.imread(img_path)
        hh, ww = img.shape[:2]

        zoomed_img = zoom_image(img, scale=3.0)

        blue_mask = createBlueMask(zoomed_img)
        yellow_mask = createYellowMask(zoomed_img)

        min_area = 1200
        max_area = 6800

        draw_and_label_boxes(zoomed_img, blue_mask, "traffic sign (blue)", min_area, max_area)
        draw_and_label_boxes(zoomed_img, yellow_mask, "traffic sign (yellow)", min_area, max_area)

        zoomed_h, zoomed_w = zoomed_img.shape[:2]
        scale_factor_h, scale_factor_w = hh / zoomed_h, ww / zoomed_w
        contours_blue, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_yellow, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        annotations = extract_annotations_for_image(filename)
        detection_ious = []

        for cnt in contours_blue:
            area = cv2.contourArea(cnt)
            if min_area <= area <= max_area:
                x, y, w, h = cv2.boundingRect(cnt)
                x_original = int(x * scale_factor_w)
                y_original = int(y * scale_factor_h)
                w_original = int(w * scale_factor_w)
                h_original = int(h * scale_factor_h)
                if not is_rectangular((x_original, y_original, w_original, h_original)):
                    cv2.rectangle(img, (x_original, y_original), (x_original + w_original, y_original + h_original), (0, 255, 0), 2)
                    cv2.putText(img, "traffic sign (blue)", (x_original, y_original - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                detected_box = (x_original, y_original, w_original, h_original)
                for annotation_box,annotation_label in annotations:
                    iou = calculate_iou(annotation_box, detected_box)
                    detection_ious.append(iou)

        for cnt in contours_yellow:
            area = cv2.contourArea(cnt)
            if min_area <= area <= max_area:
                x, y, w, h = cv2.boundingRect(cnt)
                x_original = int(x * scale_factor_w)
                y_original = int(y * scale_factor_h)
                w_original = int(w * scale_factor_w)
                h_original = int(h * scale_factor_h)
                if not is_rectangular((x_original, y_original, w_original, h_original)):
                    cv2.rectangle(img, (x_original, y_original), (x_original + w_original, y_original + h_original), (0, 255, 0), 2)
                    cv2.putText(img, "traffic sign (yellow)", (x_original, y_original - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                detected_box = (x_original, y_original, w_original, h_original)
                for annotation_box,annotation_label in annotations:
                    iou = calculate_iou(annotation_box, detected_box)
                    detection_ious.append(iou)
                    
        for (x, y, w, h), label in annotations:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        tp_image = sum([1 for iou in detection_ious if iou >= iou_threshold])
        fp_image = len(detection_ious) - tp_image
        fn_image = len(annotations) - sum([1 for iou in detection_ious if iou >= iou_threshold])

        tp_list.append(tp_image)
        fp_list.append(fp_image)
        fn_list.append(fn_image)

        if detection_ious:
            mean_iou_binary = [1 if iou >= iou_threshold else 0 for iou in detection_ious]
            mean_iou = np.mean(mean_iou_binary)
            mean_ious.append(mean_iou)

        # plt.figure(figsize=(10, 10))
        # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        # plt.axis('off')
        # plt.show()

tp = sum(tp_list)
fp = sum(fp_list)
fn = sum(fn_list)

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = (2 * precision * recall) / (precision + recall)
total_mean_iou = np.mean(mean_ious)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print(f"mIOU for all images: {total_mean_iou:.4f}")

image_path2 = r"D:\Code\Traffic\Set1Part0\1277385864Image000007.jpg"
detect_traffic_signs_with_border(image_path2)

Processing Images: 100%|██████████| 1970/1970 [07:24<00:00,  4.43it/s]

Precision: 0.06877244876399746
Recall: 0.2054275796781319
F1 Score: 0.10304709141274239
mIOU for all images: 0.1310
